# Colormaps

NiiVue provides many [colormaps](https://cmap-docs.readthedocs.io/en/latest/catalog/) to convert voxel intensity to a color range. Many of these colormaps are designed to be distinctive for individuals with color vision deficiency. This Python demo that replicates the functionality of the [colormaps web page](https://niivue.github.io/niivue/features/colormaps.html).

In [ ]:
import json
import ipywidgets as widgets
from IPython.display import display
from ipyniivue import NiiVue

# Create NiiVue instance
nv = NiiVue(back_color=(0.3, 0.3, 0.3, 1))

nv.load_volumes([{"path": "../images/mni152.nii.gz"}])

nv.opts.is_colorbar = True

## Controls

# Invert Checkbox
invert_check = widgets.Checkbox(value=False, description="Invert")


def on_invert_change(change):
    """Set colormap invert."""
    nv.volumes[0].colormap_invert = change["new"]


invert_check.observe(on_invert_change, names="value")

# Green Dragging Selection Checkbox
select_check = widgets.Checkbox(value=False, description="Green Dragging Selection")


def on_select_change(change):
    """Set selection box color."""
    if change["new"]:
        nv.set_selection_box_color((0, 1, 0, 0.7))
    else:
        nv.set_selection_box_color((1, 1, 1, 0.5))


select_check.observe(on_select_change, names="value")

# Green Crosshairs Checkbox
cross_check = widgets.Checkbox(value=False, description="Green Crosshairs")


def on_cross_change(change):
    """Set crosshair color."""
    if change["new"]:
        nv.set_crosshair_color((0, 1, 0, 1))
    else:
        nv.set_crosshair_color((1, 0, 0, 1))


cross_check.observe(on_cross_change, names="value")

# Wide Crosshairs Checkbox
wide_check = widgets.Checkbox(value=False, description="Wide Crosshairs")


def on_wide_change(change):
    """Set crosshair width."""
    if change["new"]:
        nv.set_crosshair_width(3)
    else:
        nv.set_crosshair_width(1)


wide_check.observe(on_wide_change, names="value")

# Gamma Slider
gamma_slider = widgets.IntSlider(
    value=100,
    min=10,
    max=400,
    step=10,
    description="Gamma",
    readout=False
)


def on_gamma_change(change):
    """Set gamma."""
    nv.set_gamma(change["new"] * 0.01)


gamma_slider.observe(on_gamma_change, names="value")

# Save Bitmap Button
save_bmp_button = widgets.Button(description="Save Bitmap")


def on_save_bmp_clicked(b):
    """Save scene."""
    nv.save_scene("ScreenShot.png")

save_bmp_button.on_click(on_save_bmp_clicked)

# colormap dropdown

# Dropdown for Colormaps
colormaps = sorted(nv.colormaps())
colormap_dropdown = widgets.Dropdown(
    options=colormaps,
    description="Colormap:",
    value="gray",
)

def on_colormap_change(change):
    """Handle selection in Colormap Dropdown."""
    selected_cmap = change["new"]
    if nv.volumes:
        nv.set_colormap(nv.volumes[0].id, selected_cmap)

# Organize header controls
header_controls = widgets.HBox(
    [
        invert_check,
        select_check,
        cross_check,
        wide_check,
        gamma_slider,
        save_bmp_button
    ]
)


# Custom Colormap TextArea
cmap_textarea = widgets.Textarea(
    value="""{
  "R": [0, 255, 0],
  "G": [0, 0, 255],
  "B": [0, 0, 0],
  "A": [0, 64, 64],
  "I": [0, 85, 255]
}""",
    description="Custom Colormap",
    layout=widgets.Layout(width="60%", height="100px"),
    style={"description_width": "initial"},
)

# Custom Colormap Button
custom_button = widgets.Button(description="Custom")


def on_custom_clicked(b):
    """Add the custom colormap to nv."""
    cmap = json.loads(cmap_textarea.value)
    key = "Custom"
    nv.add_colormap(key, cmap)
    nv.set_colormap(nv.volumes[0].id, key)



custom_button.on_click(on_custom_clicked)
colormap_dropdown.observe(on_colormap_change, names="value")

## Display everything

# Display header controls
display(header_controls)
# Display image
display(nv)
# Display colormap footer
display(widgets.Box([cmap_textarea, custom_button, colormap_dropdown]))
